In [ ]:
 %pylab inline

In [1]:
from pylab import *
from scipy import *
from scipy import stats, io
import numpy as np
import struct
import tables as tb
from attrdict import AttrDict
import matplotlib.pyplot as plt
import os as os
from phy.io import KwikModel
import codecs as codecs
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick

# FUNCTIONS TO BUILD PSTHs STAs AND STCs

In [ ]:
#----------------------------------------------------------------------------------------
# READ STIMULUS
#----------------------------------------------------------------------------------------
# Here we read the binary file with stimulus: 902 of 25 piezos x 1024 samples
# Text file has the type of stimulus: F sparse, C correlated, U uncorrelated
def read_stimulus(expe,meas):
    ## This function reads the stimulus binary file, reads the type of stimulus,
    ## and stores it in a matrix and a row vector
    
    bin_file = open(binname,'rb')
    read_data = np.fromfile(file=bin_file, dtype=np.float32)
    read_data = read_data.reshape((-1,25,10240)) # reshapes data assuming 25 whiskers, 10240 time bins
    txt_data = np.loadtxt(textname, dtype='S8') # Makes sure data type is text decoded
    txt_data = txt_data.view(np.chararray).decode('utf-8') # Makes sure data type is text decoded
    
    length2=0
    
    if expe==22 and meas[0:2] == 'm1':  #measurement 1 composed of two recorded
        length1=902
        length2 =93
    
    if expe == 20 and meas[0:2]=='m3': #there are 46 events in exp2 and 879 in exp3
        length1 = 46 
        length2 = 879
    
    if length2>0:
        txt_data = np.append(txt_data[0:length1], txt_data[0:length2])
        read_data = np.append(read_data[0:length1], read_data[0:length2],0)
        read_data = read_data.reshape((-1,25,10240)) # reshapes data assuming 25 whiskers, 10240 time bins
            
    close(binname)
    
    return read_data, txt_data

#----------------------------------------------------------------------------------------
# READKWIKINFO
#----------------------------------------------------------------------------------------
# We read the data of the output from klusterkwik: spike times and cluster-number of each
# cluster-number is in klustaviewa series (can be as high as 130 e.g.)
# Grupete stands for cluster groups! 2: good clusters, 1: multiunits, 0: unsorted, 3: noise
def readkwikinfo(kwik, grupete=3):
    model = KwikModel(kwik) # load kwik model from file
    spiketimes = model.spike_times # extract the absolute spike times
    clusters = model.cluster_groups # extract the cluster names
    sample_rate = model.sample_rate # extract sampling freq
    
    spikedata = {} # initialise dictionary
    for cluster in clusters.keys():
        clustergroup = clusters[cluster]
        if clustergroup==grupete: # only look at specified type of cluster, 0 = noise, 1 = MUA, 2 = GOOD, 3 = unsorted
            spiketimematrix = AttrDict({'spike_times': np.zeros(len(spiketimes[where(model.spike_clusters==cluster)]))})
            spiketimematrix.spike_times = spiketimes[where(model.spike_clusters==cluster)]
            spikedata[cluster] = spiketimematrix # data structure is a dictionary with attribute accessible spiketimes
            # attribute accessible means that spikedata.spike_times works, normal dictionaries would be spikedata[spike_times]
    
    model.close()
    return spikedata, sample_rate

#----------------------------------------------------------------------------------------
# READ VTAG
#----------------------------------------------------------------------------------------
def readVtag(Vtag1,stim,stimtype):
    ## The first task is to find the stimulus onset times for each whisker in each sweep in each direction
    start_and_stops = Vtag1[1:] - Vtag1[:-1]
    starts = (where(start_and_stops==1)[0]-2999)/float(sampling_freq) # time in seconds
    stops = (where(start_and_stops==-1)[0]+4110)/float(sampling_freq) # time in seconds
    
    stim_ret = stim[0:len(stops),:,:]
    stimtype_ret = stimtype[0:len(stops)]
    
    return stim_ret, stimtype_ret, starts, stops

#----------------------------------------------------------------------------------------
# BUILDS PSTH
#----------------------------------------------------------------------------------------
def BuildPSTH(stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops) :
## The first task is to find the stimulus onset times for each whisker in each sweep in each direction
    #stim, stimtype = read_stimulus()
    stim = stim[np.where(stimtype=='F')[0], :, :]
    starts = starts[np.where(stimtype=='F')[0]]
    stops = stops[np.where(stimtype=='F')[0]]
    
    stimtimes = {}
    for w in np.arange(25, dtype='int') :  
        timesUP = []
        timesDOWN = []
        for i in np.arange(len(stim), dtype='int') :
            indsUP = (np.where(stim[i, w, :]==1108.8889)[0]-1)[::2]
            # This finds all time points where the stim = 1108.8889, because each ramp has two 1108.8889 values
            # (on the way up and on the way down) we take every other index using [::2]
            timesUP.append(indsUP)
            indsDOWN = (np.where(stim[i, w, :]==-1108.8889)[0]-1)[::2]
            # This finds all time points where the stim = -1108.8889, because each ramp has two -1108.8889 values
            # (on the way up and on the way down) we take every other index using [::2]
            timesDOWN.append(indsDOWN)
        stimtimes[w] = timesUP, timesDOWN # stimtimes[whisker][0][:]=UP stimtimes[whisker][1][:]=DOWN
    
    # make an 'output dict'
    # the PSTH will be built on -tbefore:tafter
    hist_inds = {}
    PSTH_spike_counts = {}
    
    # Loop each neuron and get the spikes.
    for neuron in Spikes.keys(): 
        PSTH_spike_counts[neuron], hist_inds[neuron] = PSTH_spikes(stim, stimtype, stimtimes, Spikes[neuron].spike_times, sampling_freq, t_before, t_after, starts, stops)
    
    return PSTH_spike_counts, hist_inds

#----------------------------------------------------------------------------------------

def PSTH_spikes(stimulation, stimtype, stimtimes, spikes, samp, t_before, t_after, starts, stops):
    """
    stimulation   : a list of numpy arrays with a n*t stimulus inside
    stimtimes     : a list of the times the stimulus occurred for each whisker 
    spikes        : an array that contains the spike times (s)
    Vtag1         : synchronises stimulus with spike times
    samp          : sampling rate of the stimulation (Hz)
    t_before      : duration before the stim (positive, s)
    t_after       : duration after the stim (positive, s)
    starts        : the start of the F sweeps
    stops         : the stops of the F sweeps
    """
    
    stim_samp = 1/.0009997575757
    
    PSTH_spike_counts = {}
    for w in np.arange(25, dtype='int') :
        spikecountsup = 0
        spikecountsdown = 0
        for i in np.arange(len(stimulation), dtype='int') : 
            for x in np.arange(len(stimtimes[w][0][i]), dtype='int') :  # we must look at the number of stimulations per whisker per stimulation block and this is no longer 4        
                timesUP = starts[i] + stimtimes[w][0][i][x]/stim_samp # stimtimes is now stimtimes[whisker][0 for UP, 1 for DOWN][stimsweep][trial]
                spikecountsup += len(spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)]) # count spikes that are within PSTH window of stimtimes
            for y in np.arange(len(stimtimes[w][1][i]), dtype='int') : # for the DOWN stimuli we must have a separate loop because they also are now randomly distributed and not 4
                timesDOWN = starts[i] + stimtimes[w][1][i][y]/stim_samp                
                spikecountsdown += len(spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)])
        PSTH_spike_counts[w] = spikecountsup, spikecountsdown
    
    hist_inds = {} #same changes for this block, each loop will change length depending on how many stimulations fall in a sweep
    for w in np.arange(25, dtype='int') :
        hist_inds[w] = np.zeros(PSTH_spike_counts[w][0]), np.zeros(PSTH_spike_counts[w][1])
        spikecountsup = 0
        spikecountsdown = 0
        for i in np.arange(len(stimulation), dtype='int') : 
            for x in np.arange(len(stimtimes[w][0][i]), dtype='int') :     # dynamic loop depends on how many stims fall in sweep      
                timesUP = starts[i] + stimtimes[w][0][i][x]/stim_samp
                spikecountup = len(spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)])
                spikeidxup = spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)]
                spikeidxup = ((spikeidxup - starts[i])/float(stops[i] - starts[i])*len(stimulation[i,0]))
                hist_inds[w][0][spikecountsup:(spikecountsup+spikecountup)] = spikeidxup-stimtimes[w][0][i][x]
                spikecountsup += spikecountup
            
            for y in np.arange(len(stimtimes[w][1][i]), dtype='int') :     # dynamic loop depends on how many stims fall in sweep
                timesDOWN = starts[i] + stimtimes[w][1][i][y]/stim_samp                
                spikecountdown = len(spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)])
                spikeidxdown = spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)]
                spikeidxdown = ((spikeidxdown - starts[i])/float(stops[i] - starts[i])*len(stimulation[i,0]))
                hist_inds[w][1][spikecountsdown:(spikecountsdown+spikecountdown)] = spikeidxdown-stimtimes[w][1][i][y]
                spikecountsdown += spikecountdown
                
    return PSTH_spike_counts, hist_inds
# March 25 we took out the rounding fronm this function, to keep the actual times with respect to stim.
#----------------------------------------------------------------------------------------

def GetWhiskers(histdata, t_before, t_after, thresh,activity,chancelevel,start_win,end_win) :
    STC_on = {}
    PW = {}
    ActMod2={}
    ActModout={}
    
    stim_samp = 1/.0009997575757 
    before_index = int(np.around(t_before*stim_samp)) 
    after_index = int(np.around(t_after*stim_samp)) 
    bins = before_index + after_index
    
    for neuron in histdata.keys() :
        ActMod = np.zeros(25)
        Countinout = np.zeros(25)
        Count=np.zeros(25)
        SHARP = np.zeros(25)
        for i in np.arange(25, dtype='int') :
            after = 0
            before = 0
            countin = 0
            countout =0
            
            for j in np.arange(len(histdata[neuron][i][0]), dtype='int') :
                if histdata[neuron][i][0][j]>start_win-1 and histdata[neuron][i][0][j]<end_win : 
                    after+=1                              #conunt spikes in 30ms timewindow after stimulus
                    countin += 1
                elif histdata[neuron][i][0][j]<start_win :                
                    before+=1                             #conunt spikes in 20ms timewindow befor stimulus
                    countout += 1
            for j in np.arange(len(histdata[neuron][i][1]), dtype='int') :
                if histdata[neuron][i][1][j]>start_win-1 and histdata[neuron][i][1][j]<end_win :
                    after+=1                              
                    countin += 1
                elif histdata[neuron][i][1][j]<start_win-1:
                    before+=1
                    countout += 1
            if (after+before)==0 :
                before=1
            elif (after)>activity : 
                ActMod[i]= (  after*(before_index+start_win) - before*(end_win-start_win)  ) / (  after*(before_index+start_win) + before*(end_win-start_win)  )   #weight different time windows
                Countinout[i] = countin/(countout+1)
                condition1 = where((histdata[neuron][i][0]<20)*(histdata[neuron][i][0]>5))
                condition2 = where((histdata[neuron][i][1]<20)*(histdata[neuron][i][1]>5))
                SHARP[i] = sum( histdata[neuron][i][0][ condition1 ] ) + sum( histdata[neuron][i][1][condition2] )
            else :
                ActMod[i]=0               
            Count[i]=countin+countout
            
        ActModout[neuron]=ActMod
        ActMod2[neuron]=ActMod*Countinout*SHARP
        STC_on[neuron] = (ActMod>thresh)*(Count/sum(Count)>chancelevel)
        PW[neuron] = np.where(ActMod2[neuron]==max(ActMod2[neuron]))[0][0]
        if size(np.where(STC_on[neuron]==True))==0:
            PW[neuron]= 20
        
        
    return STC_on,PW, ActModout,ActMod2



# DISPLAYING FUNCTIONS

In [ ]:
#----------------------------------------------------------------------------------------
# DISPLAY PSTH 
#----------------------------------------------------------------------------------------
# Plot a single neuron PSTH, 25 piezos
def display_PSTH(expe,histdata, counts, t_before, t_after,fig,inner_grid,neuron,numspikesP,numspikesN,STC_on,PW) :
    stim_samp = 1/.0009997575757 
    before_index = int(np.around(t_before*stim_samp)) # indexes
    after_index = int(np.around(t_after*stim_samp)) # indexes
    histlength = before_index + after_index + 1
    
    nup = np.zeros((25,histlength-1))
    ndown = np.zeros((25,histlength-1))
    
    fig2 = figure()
    ax = fig2.add_subplot(1,1,1)
    for i in range(25) :
        if histdata[i][0].size :
            n1, bins, patches = ax.hist(histdata[i][0], bins = np.linspace(-before_index, after_index, histlength))
            nup[i,:] = n1
            close()
        if histdata[i][1].size :
            n2=2
            n2, bins, patches = ax.hist(histdata[i][1], bins = np.linspace(-before_index, after_index, histlength))
            ndown[i,:] = n2
            close()
    normnum = (1/np.sum(nup+ndown))
    height = np.max(np.array([np.max(nup), np.max(ndown)]))/(1/normnum)
        
    clf()
    for j in range(25) : #I use a dummy variable to sort whisker problems in exp23
        i=j
        ax1 = Subplot(fig, inner_grid[i])
        #-------------------------------------------------------
        #-------------------------------------------------------
        #for whisker problems
        #-------------------------------------------------------
        #Only for EXP 23: I shift first row to the left and leave A4 empty
        if expe == 23:
            if j<4: i=j+1
            elif j==4: continue
            elif j>4:i=j
        #-------------------------------------------------------
        #Only for EXP 27: D1 whisker missing
        if expe == 27:     
            if j==16: continue
        #-------------------------------------------------------
        #-------------------------------------------------------
        #-------------------------------------------------------
        #Only for EXP 22 or less: odd ELPHY piezo assignement
        if expe <= 22:     
            if j==11: i=24
            if j==20: i=11    
            if j>20: i=j-1
        #-------------------------------------------------------
        #-------------------------------------------------------              
                      
        if i == 0 :
            #ax1 = subplot(5,5,1, frame_on=False)
            ax1 = Subplot(fig, inner_grid[i],sharex=ax1,sharey=ax1)     
            ax1.set_xticks([])
            ax1.set_yticks([])
        elif i==20:
            ax1 = Subplot(fig,inner_grid[i],sharex=ax1,sharey=ax1)
            ax1.spines['right'].set_linewidth(0.3)
            ax1.spines['top'].set_linewidth(0.3)
            ax1.spines['left'].set_linewidth(0.3)
            ax1.spines['bottom'].set_linewidth(0.3)
            
            ax1.set_xticks([])
            ax1.set_yticks([])
        else :
            #subplot(5,5,i+1,sharex=ax1,sharey=ax1,frame_on=False)
            ax1 = Subplot(fig,inner_grid[i],sharex=ax1,sharey=ax1)
            ax1.set_xticks([])
            ax1.set_yticks([])
                                
        if PW==i:
            ax1.set_axis_bgcolor('#dddddd')    
        elif i!=20:
            ax1.spines['right'].set_visible(False)
            ax1.spines['top'].set_visible(False)
            ax1.spines['left'].set_visible(False)
            ax1.spines['bottom'].set_visible(False)
        if STC_on[i]== True:
            ax1.set_axis_bgcolor('#dddddd')    
                        
        
        if histdata[i][1].size :
            ax1.hist(histdata[i][1], bins = np.linspace(-before_index, after_index, histlength), color='g', alpha=1.0, edgecolor='none', histtype='stepfilled', label='Pos', weights=np.repeat(normnum, len(histdata[i][1])))
        if histdata[i][0].size :
            ax1.hist(histdata[i][0], bins = np.linspace(-before_index, after_index, histlength), color='b', alpha=0.7, edgecolor='none', histtype='stepfilled', label='Neg', weights=np.repeat(normnum, len(histdata[i][0]))) 
        #if (histdata[i][0].size) or (histdata[i][1].size) :
        xlim(-before_index, after_index)
        ax1.axvline(0, color = 'r', linewidth=1)
        ax1.axhline(0, color = 'r', linewidth=2)
        ymax = 1.02 * height
        ylim(0, ymax)
        xvals = np.array([0,10,20,30])
        yvals = np.array([0,ymax*0.9,ymax*0.9,0])
        ax1.plot(xvals, yvals, linewidth=0.2,color = (0.75,0.75,0.75))
        if i==4: ax1.set_title('ymax =' + str( np.around(height,decimals = 3) ),fontsize=8)
        if i ==1: ax1.set_title('Nrn' + str(neuron) + '_Pos' + str(int(numspikesP))+ '_Neg' + str(int(numspikesN)),fontsize=9)
        
        fig.add_subplot(ax1)

#----------------------------------------------------------------------------------------
def display_all_PSTHs_of_recording(expe,histdata, counts, pdf_files_directory, t_before, t_after,grupete,STC_on,PW,titles) :
    
    fig = plt.figure(figsize=(12,16.5))
    nrns = len(histdata.keys())
    if nrns <16: 
        layout = [5,3]
    else: layout = [nrns//3+(nrns%3!=0),3]
    outer_grid = gridspec.GridSpec(layout[0], layout[1], wspace=0.1, hspace=0.2)
    
    ii=0
    orderneurons = np.sort(list(histdata.keys()))
    for neuron in  orderneurons:
        clf()
        totalup = 0
        totaldown = 0
        for i in np.arange(25, dtype='int') :
            totalup+=counts[neuron][i][0]
            totaldown+=counts[neuron][i][1]
            
        inner_grid = gridspec.GridSpecFromSubplotSpec(5,5,subplot_spec=outer_grid[ii], wspace=0.1, hspace=0.1)
               
        numspikesP= totalup                  
        numspikesN= totaldown
        display_PSTH(expe,histdata[neuron], counts[neuron], t_before, t_after,fig,inner_grid,neuron,numspikesP,numspikesN,STC_on[neuron],PW[neuron])                               
        
        if grupete ==1:
            fig.suptitle(titles + '_multiunits',fontsize=16)
        elif grupete ==3:
            fig.suptitle(titles + '_responsiveMULTIUNITS',fontsize=16)
        else:
            fig.suptitle(titles ,fontsize=16)
        
        ii+=1
    if grupete ==1:                  
        fig.savefig(pdf_files_directory + titles + '_hist_multi.pdf', format='pdf')
    elif grupete==3:
        fig.savefig(pdf_files_directory + titles + '_hist_respMULTI.pdf', format='pdf')
    else:        
        fig.savefig(pdf_files_directory + titles + '_hist.pdf', format='pdf')
        
    clf()



# Experiment Files and Folders

In [ ]:
# In this cell you put all the information to make the code portable from computer to computer
# You have to place all the file names and experiments, then you loop whichever you want to analyse
#--------------------------------------------------------------------------------
#Experiment numbers
ExpeNum = [20,22,23,24,25,26,27,28,29,30,31,32]

#--------------------------------------------------------------------------------
#Folders for measurements and experiments (this is how we separate shanks in folders for individual analyses)
m164 = ['m1s1','m1s2','m1s3','m1s4','m1s5','m1s6','m1s7','m1s8']
m264 = ['m2s1','m2s2','m2s3','m2s4','m2s5','m2s6','m2s7','m2s8']
m364 = ['m3s1','m3s2','m3s3','m3s4','m3s5','m3s6','m3s7','m3s8']
m464 = ['m4s1','m4s2','m4s3','m4s4','m4s5','m4s6','m4s7','m4s8']

#--------------------------------------------------------------------------------
#Kwik files    

files20 = [ 'MEAS-150707-1_ele01_ele08.kwik',
            'MEAS-150707-1_ele09_ele16.kwik',
            'MEAS-150707-1_ele17_ele24.kwik',
            'MEAS-150707-1_ele25_ele32.kwik',
            'MEAS-150707-23_ele01_ele08.kwik',
            'MEAS-150707-23_ele16_ele09.kwik',
            'MEAS-150707-23_ele17_ele24.kwik',
            'MEAS-150707-23_ele25_ele32.kwik',]

files22 = [ 'MEAS-150716-12_ele01_ele08.kwik',
            'MEAS-150716-12_ele09_ele16.kwik',
            'MEAS-150716-12_ele17_ele24.kwik',
            'MEAS-150716-12_ele25_ele32.kwik',
            'MEAS-150716-3_ele01_ele08.kwik',
            'MEAS-150716-3_ele09_ele16.kwik',
            'MEAS-150716-3_ele17_ele24.kwik',
            'MEAS-150716-3_ele25_ele32.kwik',]

files23 = [ 'MEAS-151027-1_ele01_ele08.kwik',
            'MEAS-151027-1_ele09_ele16.kwik',
            'MEAS-151027-1_ele17_ele24.kwik',
            'MEAS-151027-1_ele25_ele32.kwik',
            'MEAS-151027-1_ele33_ele40.kwik',
            'MEAS-151027-1_ele41_ele48.kwik',
            'MEAS-151027-1_ele49_ele56.kwik',
            'MEAS-151027-1_ele57_ele64.kwik',
            'MEAS-151027-2_ele01_ele08.kwik',
            'MEAS-151027-2_ele09_ele16.kwik',
            'MEAS-151027-2_ele17_ele24.kwik',
            'MEAS-151027-2_ele25_ele32.kwik',
            'MEAS-151027-2_ele33_ele40.kwik',
            'MEAS-151027-2_ele41_ele48.kwik',
            'MEAS-151027-2_ele49_ele56.kwik',
            'MEAS-151027-2_ele57_ele64.kwik']

files24 = ['MEAS-151103-1_EXTRACTED_ele25_ele32.kwik',
           'MEAS-151103-1_EXTRACTED_ele33_ele40.kwik',
           'MEAS-151103-1_EXTRACTED_ele41_ele48.kwik',
           'MEAS-151103-1_EXTRACTED_ele49_ele56.kwik',
           'MEAS-151103-1_EXTRACTED_ele57_ele64.kwik',
           'MEAS-151103-2_ele33_ele40.kwik',
           'MEAS-151103-2_ele41_ele48.kwik',
           'MEAS-151103-2_ele49_ele56.kwik',
           'MEAS-151103-2_ele57_ele64.kwik']

files25 = [ 'MEAS-151105-1good_ele01_ele08.kwik',
            'MEAS-151105-1good_ele09_ele16.kwik',
            'MEAS-151105-1good_ele17_ele24.kwik',
            'MEAS-151105-1good_ele25_ele32.kwik',
            'MEAS-151105-1good_ele33_ele40.kwik',
            'MEAS-151105-1good_ele41_ele48.kwik',
            'MEAS-151105-1good_ele49_ele56.kwik',
            'MEAS-151105-1good_ele57_ele64.kwik',
            'MEAS-151105-2_ele01_ele08.kwik',
            'MEAS-151105-2_ele09_ele16.kwik',
            'MEAS-151105-2_ele17_ele24.kwik',
            'MEAS-151105-2_ele25_ele32.kwik',
            'MEAS-151105-2_ele33_ele40.kwik',
            'MEAS-151105-2_ele41_ele48.kwik',
            'MEAS-151105-2_ele49_ele56.kwik',
            'MEAS-151105-2_ele57_ele64.kwik']

files26 = [ 'MEAS-151110-1_ele01_ele08.kwik',
            'MEAS-151110-1_ele09_ele16.kwik',
            'MEAS-151110-1_ele17_ele24.kwik',
            'MEAS-151110-1_ele25_ele32.kwik',
            'MEAS-151110-1_ele33_ele40.kwik',
            'MEAS-151110-1_ele41_ele48.kwik',
            'MEAS-151110-1_ele49_ele56.kwik',
            'MEAS-151110-1_ele57_ele64.kwik',
            'MEAS-151110-2_ele01_ele08.kwik',
            'MEAS-151110-2_ele09_ele16.kwik',
            'MEAS-151110-2_ele17_ele24.kwik',
            'MEAS-151110-2_ele25_ele32.kwik',
            'MEAS-151110-2_ele33_ele40.kwik',
            'MEAS-151110-2_ele41_ele48.kwik',
            'MEAS-151110-2_ele49_ele56.kwik',
            'MEAS-151110-2_ele57_ele64.kwik',
            'MEAS-151110-3_ele01_ele08.kwik',
            'MEAS-151110-3_ele09_ele16.kwik',
            'MEAS-151110-3_ele17_ele24.kwik',
            'MEAS-151110-3_ele25_ele32.kwik',
            'MEAS-151110-3_ele33_ele40.kwik',
            'MEAS-151110-3_ele41_ele48.kwik',
            'MEAS-151110-3_ele49_ele56.kwik',
            'MEAS-151110-3_ele57_ele64.kwik']

files27  = ['MEAS-151112-1_ele01_ele08.kwik',
            'MEAS-151112-1_ele09_ele16.kwik',
            'MEAS-151112-1_ele17_ele24.kwik',
            'MEAS-151112-1_ele25_ele32.kwik',
            'MEAS-151112-1_ele33_ele40.kwik',
            'MEAS-151112-1_ele41_ele48.kwik',
            'MEAS-151112-1_ele49_ele56.kwik',
            'MEAS-151112-1_ele57_ele64.kwik',
            'MEAS-151112-2_ele01_ele08.kwik',
            'MEAS-151112_2_ele09_ele16.kwik',
            'MEAS-151112-2_ele17_ele24.kwik',
            'MEAS-151112-2_ele25_ele32.kwik',
            'MEAS-151112-2_ele33_ele40.kwik',
            'MEAS-151112-2_ele41_ele48.kwik',
            'MEAS-151112-2_ele49_ele56.kwik',
            'MEAS-151112-2_ele57_ele64.kwik',
            'MEAS-151112-3_ele01_ele08.kwik',
            'MEAS-151112-3_ele09_ele16.kwik',
            'MEAS-151112-3_ele17_ele24.kwik',
            'MEAS-151112-3_ele25_ele32.kwik',
            'MEAS-151112-3_ele33_ele40.kwik',
            'MEAS-151112-3_ele41_ele48.kwik',
            'MEAS-151112-3_ele49_ele56.kwik',
            'MEAS-151112-3_ele57_ele64.kwik']

files28 =  ['MEAS-151116-1_ele01_ele08.kwik',
            'MEAS-151116-1_ele09_ele16.kwik',
            'MEAS-151116-1_ele17_ele24.kwik',
            'MEAS-151116-1_ele25_ele32.kwik',
            'MEAS-151116-1_ele33_ele40.kwik',
            'MEAS-151116-1_ele41_ele48.kwik',
            'MEAS-151116-1_ele49_ele56.kwik',
            'MEAS-151116-1_ele57_ele64.kwik',
            'MEAS-151116-2_ele01_ele08.kwik',
            'MEAS-151116-2_ele09_ele16.kwik',
            'MEAS-151116-2_ele17_ele24.kwik',
            'MEAS-151116-2_ele25_ele32.kwik',
            'MEAS-151116-2_ele33_ele40.kwik',
            'MEAS-151116-2_ele41_ele48.kwik',
            'MEAS-151116-2_ele49_ele56.kwik',
            'MEAS-151116-2_ele57_ele64.kwik',
            'MEAS-151116-3_ele01_ele08.kwik',
            'MEAS-151116-3_ele09_ele16.kwik',
            'MEAS-151116-3_ele17_ele24.kwik',
            'MEAS-151116-3_ele25_ele32.kwik',
            'MEAS-151116-3_ele33_ele40.kwik',
            'MEAS-151116-3_ele41_ele48.kwik',
            'MEAS-151116-3_ele49_ele56.kwik',
            'MEAS-151116-3_ele57_ele64.kwik']

files29  = ['MEAS-151118-1_ele01_ele08.kwik',
            'MEAS-151118-1_ele09_ele16.kwik',
            'MEAS-151118-1_ele17_ele24.kwik',
            'MEAS-151118-1_ele25_ele32.kwik',
            'MEAS-151118-1_ele33_ele40.kwik',
            'MEAS-151118-1_ele41_ele48.kwik',
            'MEAS-151118-1_ele49_ele56.kwik',
            'MEAS-151118-1_ele57_ele64.kwik',
            'MEAS-151118-2_ele01_ele08.kwik',
            'MEAS-151118-2_ele09_ele16.kwik',
            'MEAS-151118-2_ele17_ele24.kwik',
            'MEAS-151118-2_ele25_ele32.kwik',
            'MEAS-151118-2_ele33_ele40.kwik',
            'MEAS-151118-2_ele41_ele48.kwik',
            'MEAS-151118-2_ele49_ele56.kwik',
            'MEAS-151118-2_ele57_ele64.kwik',
            'MEAS-151118-3_ele01_ele08.kwik',
            'MEAS-151118-3_ele09_ele16.kwik',
            'MEAS-151118-3_ele17_ele24.kwik',
            'MEAS-151118-3_ele25_ele32.kwik',
            'MEAS-151118-3_ele33_ele40.kwik',
            'MEAS-151118-3_ele41_ele48.kwik',
            'MEAS-151118-3_ele49_ele56.kwik',
            'MEAS-151118-3_ele57_ele64.kwik']


files30  = ['MEAS-151208-2_ele01_ele08.kwik',
            'MEAS-151208-2_ele09_ele16.kwik',
            'MEAS-151208-2_ele17_ele24.kwik',
            'MEAS-151208-2_ele25_ele32.kwik',
            'MEAS-151208-2_ele33_ele40.kwik',
            'MEAS-151208-2_ele41_ele48.kwik',
            'MEAS-151208-2_ele49_ele56.kwik',
            'MEAS-151208-2_ele57_ele64.kwik',
            'MEAS-151208-3_ele01_ele08.kwik',
            'MEAS-151208-3_ele09_ele16.kwik',
            'MEAS-151208-3_ele17_ele24.kwik',
            'MEAS-151208-3_ele25_ele32.kwik',
            'MEAS-151208-3_ele33_ele40.kwik',
            'MEAS-151208-3_ele41_ele48.kwik',
            'MEAS-151208-3_ele49_ele56.kwik',
            'MEAS-151208-3_ele57_ele64.kwik',
            'MEAS-151208-4_ele01_ele08.kwik',
            'MEAS-151208-4_ele09_ele16.kwik',
            'MEAS-151208-4_ele17_ele24.kwik',
            'MEAS-151208-4_ele25_ele32.kwik',
            'MEAS-151208-4_ele33_ele40.kwik',
            'MEAS-151208-4_ele41_ele48.kwik',
            'MEAS-151208-4_ele49_ele56.kwik',
            'MEAS-151208-4_ele57_ele64.kwik',
            'MEAS-151208-5_ele01_ele08.kwik',
            'MEAS-151208-5_ele09_ele16.kwik',
            'MEAS-151208-5_ele17_ele24.kwik',
            'MEAS-151208-5_ele25_ele32.kwik',
            'MEAS-151208-5_ele33_ele40.kwik',
            'MEAS-151208-5_ele41_ele48.kwik',
            'MEAS-151208-5_ele49_ele56.kwik',
            'MEAS-151208-5_ele57_ele64.kwik']


files31 = [ 'MEAS-151210-1_ele01_ele08.kwik',
            'MEAS-151210-1_ele09_ele16.kwik',
            'MEAS-151210-1_ele17_ele24.kwik',
            'MEAS-151210-1_ele25_ele32.kwik',
            'MEAS-151210-1_ele33_ele40.kwik',
            'MEAS-151210-1_ele41_ele48.kwik',
            'MEAS-151210-1_ele49_ele56.kwik',
            'MEAS-151210-1_ele57_ele64.kwik',
            'MEAS-151210-2_ele01_ele08.kwik',
            'MEAS-151210-2_ele09_ele16.kwik',
            'MEAS-151210-2_ele17_ele24.kwik',
            'MEAS-151210-2_ele25_ele32.kwik',
            'MEAS-151210-2_ele33_ele40.kwik',
            'MEAS-151210-2_ele41_ele48.kwik',
            'MEAS-151210-2_ele49_ele56.kwik',
            'MEAS-151210-2_ele57_ele64.kwik',
            'MEAS-151210-3_ele01_ele08.kwik',
            'MEAS-151210-3_ele09_ele16.kwik',
            'MEAS-151210-3_ele17_ele24.kwik',
            'MEAS-151210-3_ele25_ele32.kwik',
            'MEAS-151210-3_ele33_ele40.kwik',
            'MEAS-151210-3_ele41_ele48.kwik',
            'MEAS-151210-3_ele49_ele56.kwik',
            'MEAS-151210-3_ele57_ele64.kwik']

files32 = [ 'MEAS-151214-1_ele01_ele08.kwik',
            'MEAS-151214-1_ele09_ele16.kwik',
            'MEAS-151214-1_ele17_ele24.kwik',
            'MEAS-151214-1_ele25_ele32.kwik',
            'MEAS-151214-1_ele33_ele40.kwik',
            'MEAS-151214-2_ele01_ele08.kwik',
            'MEAS-151214-2_ele09_ele16.kwik',
            'MEAS-151214-2_ele17_ele24.kwik',
            'MEAS-151214-2_ele25_ele32.kwik',
            'MEAS-151214-2_ele33_ele40.kwik']

#--------------------------------------------------------------------------------
#--------------------------------------------------------------------------------
# Here I create my dictionary of experiments
Expe={}
Vtags={}
Stim={}
for num in ExpeNum: 
    Expe[num] = dict()
    Vtags[num] = dict()
    
i=0    
for meas in np.append(m164[0:4],m364[0:4]):
    Expe[20][meas] = files20[i]
    Expe[22][meas] = files22[i]
    i+=1
i=0    
for meas in np.append(m164,m264):
    Expe[23][meas] = files23[i]
    Expe[25][meas] = files25[i]
    i+=1
i=0
for meas in np.append(m164[3:8],m264[4:8]):    
    Expe[24][meas] = files24[i]
    i+=1
i=0
for meas in np.append(np.append(m164,m264),m364):
    Expe[26][meas] = files26[i]
    Expe[27][meas] = files27[i]
    Expe[28][meas] = files28[i]
    Expe[29][meas] = files29[i]
    Expe[30][meas] = files30[i]
    Expe[31][meas] = files31[i]
    i+=1
i=0
for meas in m464:
    Expe[30][meas] = files30[i]
    i+=1
i=0
for meas in np.append(m164[0:5],m264[0:5]):
    Expe[32][meas] = files32[i]
    i+=1

#--------------------------------------------------------------------------------
#Vtag files 
Vtags[20] = ['MEAS-150707-1_Vtag1.dat','nada','MEAS-150707-23_Vtag1.dat']
Vtags[22] = ['MEAS-150716-12_Vtag1.dat','nada','MEAS-150716-3_Vtag1.dat']
Vtags[23] = ['MEAS-151027-1_Vtag1.dat','MEAS-151027-2_Vtag1.dat']
Vtags[23] = ['MEAS-151027-1_Vtag1.dat','MEAS-151027-2_Vtag1.dat']
Vtags[24] = ['MEAS-151103-1_Vtag1.dat','MEAS-151103-2_Vtag1.dat']
Vtags[25] = ['MEAS-151105-1good_Vtag1.dat','MEAS-151105-2_Vtag1.dat']
Vtags[26] = ['MEAS-151110-1_Vtag1.dat','MEAS-151110-2_Vtag1.dat','MEAS-151110-3_Vtag1.dat']
Vtags[27] = ['MEAS-151112-1_Vtag1.dat','MEAS-151112-2_Vtag1.dat','MEAS-151112-3_Vtag1.dat']
Vtags[28] = ['MEAS-151116-1_Vtag1.dat','MEAS-151116-2_Vtag1.dat','MEAS-151116-3_Vtag1.dat']
Vtags[29] = ['MEAS-151118-1_Vtag1.dat','MEAS-151118-2_Vtag1.dat','MEAS-151118-3_Vtag1.dat']
Vtags[30] = ['MEAS-151208-2_Vtag1.dat','MEAS-151208-3_Vtag1.dat','MEAS-151208-4_Vtag1.dat','MEAS-151208-5_Vtag1.dat']
Vtags[31] = ['MEAS-151210-1_Vtag1.dat','MEAS-151210-2_Vtag1.dat','MEAS-151210-3_Vtag1.dat']
Vtags[32] = ['MEAS-151214-1_Vtag1.dat','MEAS-151214-2_Vtag1.dat']

#--------------------------------------------------------------------------------
#Stimulus type
for i in range(23,33):
    Stim[i] = 'big_STIM_FC_corrected'
for i in range(15,23):
    Stim[i] = 'big_STIM'
for i in range(10,15):
    Stim[i] = 'small_STIM'  
    
#--------------------------------------------------------------------------------
#Root folder to work in, such all will be in subfolders 
#e.g.: "/EXP_23/m1s1/" for data or "/STIM/" for stims
rootF = '/home/matias/WORKSPACE/'    
stimFolder = rootF +'STIM/'

#I have a separate folder for exp 22 and before
rootF_kwiks = rootF    #uncomment this to work with the other root folder
#rootF_kwiks = '/media/matias/DATA/WORKSPACE2/'

# Define and load data files from experiments

In [ ]:
global binname, textname
#---------------------------------------------------------------------------------------
SelExp = [22]   #Expe                                        #select experiment numbers!
grupete = [2]   #select cluster groups! 2 for good clusters 1 for multiunits, 3 for unsorted

#select measurement and/or shanks!
Measurements =['m3s2']# m264[7:8]           #['m1s1']#['m3s1','m3s3']#m12[-4:]#['m1s1','m1s2','m1s3','m1s4']   

#select type of stimuli, for PSTH is only 'F' and this does not change anything
choices = ['F']                      #select stimulus type (for STA and STC)

# choice code not to ploteverything at the same time
ploteo = [1,0,0,0]                                           #1 to make plots: psth,sta,ufc,stc

dirs =[]
#--------------------------------------------------------------------------------
# Loop Experiments
#--------------------------------------------------------------------------------
last_exp=0     #we use this to load stim only when we change experiment
for expe in SelExp:
    
    Measurements = sorted(Expe[expe])                         #uncommento to select all
    print(expe)

    #This two lines are to account for diffrerent stims when looping diffrerent experiments
    binname= stimFolder + Stim[expe] + '/Stimulus_UCC.bin'     
    textname=stimFolder + Stim[expe] + '/Stimulus_UCC.txt'
    
    last_meas =0   #we use this to find when we change measurement to load Vtag and stim again
    
    #--------------------------------------------------------------------------------
    #loop goodunits, multiunits, unsorted...
    for group in grupete:   #2 for good clusters 1 for multiunits 3 for unsorted
        #folder names
        if group ==3:
            dirs  = [rootF + 'OUTPUT/PDFallRM/EXP_'+str(expe),rootF + 'OUTPUT/PDFallRM/STC/EXP_'+str(expe)]
        if group ==2:
            dirs  = [rootF + 'OUTPUT/PDFall2/EXP_'+str(expe),rootF + 'OUTPUT/PDFall2/STC/EXP_'+str(expe)]
        if group ==1:
            dirs  = [rootF + 'OUTPUT/PDFallM/EXP_'+str(expe),rootF + 'OUTPUT/PDFallM/STC/EXP_'+str(expe)]
        print('group:', group)
        #--------------------------------------------------------------------------------
        #loop measurements and shanks
        measurements = Expe[expe]                            
                 
        for meas in Measurements:           
            print(meas)
            current_meas = int(meas[1])   #measurement number
            #---------------------------------------------------------------
            #select datafile
            sp_file = rootF_kwiks + 'EXP_' + str(expe) +'/Spike_Sorting/'+ meas +'/'+ measurements[meas]
            #load datafile
            Spikes, sampling_freq = readkwikinfo(sp_file, group)  
            #---------------------------------------------------------------
            #load stimulus if looping new experiment, without trimming
            if expe!=last_exp or (expe==20 and meas=='m3s1') or (expe==22 and meas=='m3s1'):
                stimraw = []
                stimtyperaw=[]
                print("   reading stim at:",  meas)
                stimraw,stimtyperaw = read_stimulus(expe,meas)

            #---------------------------------------------------------------
            #load Vtag if looping new measurement
            if (last_meas!=current_meas): #or (expe!=last_exp):   
                # get Vtag name
                measV=int(meas[1])-1
                bin_file = rootF_kwiks + 'EXP_' + str(expe) +'/' + Vtags[expe][measV]
                #-----------------------------------------------------------
                Vtag1 =[]                
                Vtag1 = np.fromfile(file=bin_file, dtype=np.int16)
                # here we trim down stim and stimtype from Vtag1 information
                print("   trimming stim at:", meas)
                stim = []
                stimtype=[]
                stim, stimtype,starts, stops = readVtag(Vtag1,stimraw,stimtyperaw)
                                          
            last_meas = current_meas     #update measurement variable
            last_exp = expe              #update experiment variable
            
            if len(Spikes.keys())>0:                              #do only if there are clusters
                #--------------------------------------------------------------------------------
                #create output folders
                for dir in dirs:
                    if not os.path.exists(dir):
                        os.makedirs(dir) 
                dire = dirs[0] +'/'
                titles = 'Exp'+ str(expe) + '_Meas_' + meas[1] + '_Shank_' + meas[3]
                #--------------------------------------------------------------------------------
                #Build PSTHs
                t_before = .005  
                t_after = .060
                
                start_win=10  #10
                end_win=60
                
                thresh = 0.3#0.37      #Act mod activity between 10,45ms and -10,10 ms (from -1 to 1)
                activity = 40#40        #minimun amount of activity to get an active whisker from 10 to 45ms
                chancelevel = 1/25*1.40 #1/25*1.4   #how much activity above chance in response area (10-45ms)
                
                print('   building psths')
                PSTH_spikes_counts, hist_output = BuildPSTH(stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops)
                print('   finding PW and ON whiskers')
                STC_on, PW ,ActMod,ActMod2= GetWhiskers(hist_output, t_before, t_after, thresh,activity,chancelevel,start_win,end_win)
                                
                #--------------------------------------------------------------------------------
                #Plot PSTH
                print('   plotting')
                if ploteo[0]==1: display_all_PSTHs_of_recording(expe,hist_output, PSTH_spikes_counts, dire, t_before, t_after,group,STC_on,PW,titles)
            

In [ ]:
STC_on = np.zeros(25)
STC_on[0]
                

In [ ]:
PW[55]
STC_on[55]

In [2]:
#----------------------------------------------------------------------------------------
#SAVING BINARY OBJECTS DATA
#need to automate data folder creation
#----------------------------------------------------------------------------------------
import pickle 

def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
#update spiketimes to wavedataAll
SelExp = [22] #[22,24,26,27,28,29,30,31,32] #Expe                                        #select experiment numbers!

Folder = '/home/matias/WORKSPACE/S2_git/data'    

for e in SelExp:

    data = Folder +'/wavedata'+ str(e)
    wavedata = load_obj(data)

    idx = list(wavedata.keys())

    datapsth = Folder +'/psthdata' + str(e)
    psthdata = load_obj(datapsth)


    for n in idx:
        wavedata[n].update(psthdata[n])    
            
    data = Folder +'/datamix' + str(e)

    save_obj(wavedata, data)

In [23]:
SelExp = [20] #[22,24,26,27,28,29,30,31,32] #Expe                                        #select experiment numbers!

Folder = '/home/matias/WORKSPACE/S2_git/data'    

for e in SelExp:
 
    datapsth = Folder +'/psthdata' + str(e)
    psthdata = load_obj(datapsth)
    idx = list(psthdata.keys())

    print(idx)
    
    for n in idx:
 
        meas = n[6:10]
        cluster = int(n[12:])
        expe = int(n[3:5])
        psthdata[n].update(AttrDict({'clusnum': cluster,'exp' : int(expe) , 'meas': int(meas[1]) , 'shank': int(meas[3])}))


    save_obj(psthdata,datapsth)

['exp20_m1s4_c92', 'exp20_m1s4_c113', 'exp20_m1s3_c120', 'exp20_m1s4_c81', 'exp20_m1s3_c61', 'exp20_m1s4_c86', 'exp20_m1s4_c73', 'exp20_m3s2_c22', 'exp20_m3s2_c20', 'exp20_m3s2_c14', 'exp20_m1s1_c29']


In [51]:
datapsth = Folder +'/wavedata' + str(e)
psthdata = load_obj(datapsth)

idx = list(psthdata.keys())
len(idx)

print(psthdata[idx[0]].keys())

psthdata[idx[0]]

dict_keys(['shank', 'bigwavestd', 'waves', 'parnames', 'exp', 'meanwave', 'params', 'stdwave', 'electrodemax', 'meas', 'bigwave'])


AttrDict({'shank': 4, 'bigwavestd': array([  92.82623291,   95.88230133,   97.06652069,   98.11768341,
         98.47946167,   98.4016571 ,   98.18025208,   97.83772278,
         98.47612   ,  102.9906311 ,  116.92022705,  144.65878296,
        171.12242126,  169.76579285,  148.84498596,  156.0418396 ,
        173.55628967,  156.55236816,  119.15442657,  100.82888794,
         99.55000305,   95.87075043,   90.6293869 ,   87.78092194,
         87.82889557,   89.92704773,   92.29813385,   94.30775452,
         96.62130737,   99.05584717,  100.6942749 ,  100.99863434,
         99.49995422,   95.80363464,   90.39149475,   84.85958099,
         81.21976471,   80.7591629 ,   83.07585144,   86.80177307,
         90.98021698,   95.80693054,  101.65518951,  107.72384644,
        112.96903992,  116.87474823,  119.06620026,  118.80740356], dtype=float32), 'waves': [], 'parnames': ['widths0-25-50-tot-tot_int', 'cross1-2-3', 'crossrel1-2-3', 'peakVals', 'peaks1-2', 'peaksrel1-2', 'mins'], 'exp': 20

In [56]:
datapsth = Folder +'/psthdata' + str(e)
psthdata = load_obj(datapsth)

idx = list(psthdata.keys())
len(idx)

print(psthdata[idx[0]].keys())

psthdata[idx[0]]

dict_keys(['shank', 'psth_length', 'exp', 'psth_times', 'clusnum', 'psth_counts', 'meas'])


AttrDict({'shank': 4, 'psth_length': [0.1, 0.1], 'exp': 20, 'psth_times': {0: (array([-49.24118888, -44.47315637, -42.53926906, -40.53869598,
       -38.50478002, -36.17077809, -15.74956368, -13.88236214,
       -11.81510328,  -9.64781578,   1.15527886,   6.09143866,
         8.125348  ,  10.52602788,  12.59328   ,  14.827246  ,
       -63.09932533, -61.1987809 , -58.99815051, -56.99757743,
       -54.83028993, -52.86305973,   6.10560282,   8.03948384,
        10.00670763,  12.17398808,  14.24124019,  22.57693422,
        31.81288321,  53.88791748,  55.92182682,  58.1891356 ,
        60.22304494,  62.29029706,  64.52426306,  66.6915435 ,
        68.9255095 ,  48.42823326,  50.12871484,  52.16262418,
        55.13013125,  66.2332757 ,  79.04165568,  80.94219392,
        82.74270383,  87.01057917,  88.97780296,  90.97836953,
        95.6130154 ,  96.41324203, -54.4873711 , -52.65351841,
       -50.68629462, -48.81909916, -46.71850426, -44.68459492,
       -42.65068558, -38.41615301, -35.